# 2018.02.13

## 标准绘图

***Filename : CoordinateScale.py***
``` python
import stddraw

numOfLines = 50 #change the number.

stddraw.setXscale(0, numOfLines)
stddraw.setYscale(0, numOfLines)
for i in range(numOfLines + 1):
    stddraw.line(0, numOfLines - i, i, 0)
stddraw.show()
```

In [22]:
#run.
!python CoordinateScale.py

## 随机Web冲浪模型

假定每一个网页<font color=#92D050>(page)</font>中包括了规定数量的超链接<font color=#92D050>(Link)</font>，每个链接又会跳转到另一个页面。**研究的目标**为一个人随机从一个页面到另一个页面之间的行为，这个人也可以在导航栏键入网页地址跳转到下一个界面。

模型采用***90-10法则***，也就是有**90%的人会点击链接进入下一个网页**，而10%的人会输入网页网址跳转。

In [24]:
#see data in tiny.txt.
!more tiny.txt

5 
0 1 
1 2  1 2 
1 3  1 3  1 4 
2 3 
3 0 
4 0  4 2 


***Filename : transition.py***
```python
import stdio
import stdarray

# Read links from standard input and write the corresponding
# transition matrix to standard output. First, process the input
# to count the outlinks from each page. Then apply the 90-10 rule to
# compute the transition matrix. Assume that there are no pages that
# have no outlinks in the input.

n = stdio.readInt()

linkCounts = stdarray.create2D(n, n, 0)
outDegrees = stdarray.create1D(n, 0)

while not stdio.isEmpty():
    # Accumulate link counts.
    i = stdio.readInt()
    j = stdio.readInt()
    outDegrees[i] += 1
    linkCounts[i][j] += 1

stdio.writeln(str(n) + ' ' + str(n))

for i in range(n):
    # Print probability distribution for row i.
    for j in range(n):
        # Print probability for column j.
        p = (.90 * linkCounts[i][j] / outDegrees[i]) + (.10 / n)
        stdio.writef('%8.5f', p)
    stdio.writeln()
```

In [25]:
#run.
!python transition.py < tiny.txt

5 5
 0.02000 0.92000 0.02000 0.02000 0.02000
 0.02000 0.02000 0.38000 0.38000 0.20000
 0.02000 0.02000 0.02000 0.92000 0.02000
 0.92000 0.02000 0.02000 0.02000 0.02000
 0.47000 0.02000 0.47000 0.02000 0.02000


上面的运算结果很好的展示了5个页面之间的跳转关系。

| | Page1 | Page2 | Page3 | Page4 | Page5 |
| - | - | - | - | - | - |
| Page1 | 0.02000 | 0.92000 | 0.02000 | 0.02000 | 0.02000 |
| Page2 | 0.02000 | 0.02000 | 0.38000 | 0.38000 | 0.20000 |
| Page3 | 0.02000 | 0.02000 | 0.02000 | 0.92000 | 0.02000 |
| Page4 | 0.92000 | 0.02000 | 0.02000 | 0.02000 | 0.02000 |
| Page5 | 0.47000 | 0.02000 | 0.47000 | 0.02000 | 0.02000 |

接下来，如果我们正在Page1，那么从Page1到其他各个网址的概率就是如下所示了：

| | Page1 | Page2 | Page3 | Page4 | Page5 |
| - | - | - | - | - | - |
| Page1 | 0.02000 | 0.92000 | 0.02000 | 0.02000 | 0.02000 |

## 马尔科夫链

俄罗斯数学家安德雷·马尔科夫发现了这个概念，其基本定理是：**不管初始值如何，随机过程最终会趋于稳定，且继续增加随机过程不会产生新的信息，这种现象称为<font color=red>混合</font>**。但是马尔科夫链的基本应用场景就是要保证整个过程随机。

***Filename : randomsurfer.py***
```python
import stdio
import stdarray
import sys
import random

# Accept an integer moves as a command-line argument. Read a
# transition matrix from standard input. Perform moves moves as
# prescribed by the transition matrix, and write to standard output
# the relative frequency of hitting each page.

moves = int(sys.argv[1])

n = stdio.readInt()
stdio.readInt() # Discard the second int of standard input.

# Read the transition matrix from standard input.
# p[i][j] is the probability that the surfer moves from
# page i to page j.
p = stdarray.create2D(n, n, 0.0)
for i in range(n):
    for j in range(n):
        p[i][j] = stdio.readFloat()

# Perform the simulation, thus computing the hits array.
# hits[i] is the number of times the surfer hits page i.
hits = stdarray.create1D(n, 0)
page = 0  # Start at page 0.
for i in range(moves):
    # Make one random move.
    r = random.random()
    total = 0.0
    for j in range(0, n):
        # Find interval containing r.
        total += p[page][j]
        if r < total:
            page = j
            break
    hits[page] += 1

# Write the page ranks.
for v in hits:
    stdio.writef("%8.5f", 1.0 * v / moves)
stdio.writeln()
```

In [1]:
#run.
!python transition.py < tiny.txt | python randomsurfer.py 100
!python transition.py < tiny.txt | python randomsurfer.py 10000
!python transition.py < tiny.txt | python randomsurfer.py 10000000

 0.30000 0.27000 0.11000 0.26000 0.06000
 0.27250 0.26470 0.14890 0.24690 0.06700
 0.27308 0.26575 0.14614 0.24719 0.06783


基于各种原因，Web的准确页面排名在实践中具有重要价值。首先，基于页面排名网页以匹配搜索条件的Web搜索比其他方法更能满足用户的期望。其次，页面排名的可信度和可靠度导致基于页面排名的大量Web广告费用的投入。

即便在我们简单的例子中，页面排名结果也会说服那些广告商将四倍于页面4的广告费用投放在页面0上，计算机页面排名是一个数学问题，一个有趣的计算机科学难题，也是一个巨大的商机，三位一体。

## 混合马尔科夫链

由于马尔科夫链的基本极限定理，不管初始值如何，整个过程都将收敛至相同的向量，因此，原来的矩阵自乘算法就可以变成向量-矩阵乘法，大大减小了运算量。

***Filename : markov.py ***
```python
import stdio
import stdarray
import sys

# Accept integer moves from the command-line, and read a transition
# matrix from standard input. Compute the probabilities that a
# random surfer lands on each page (page ranks) after moves
# matrix-vector multiplies, and write the page ranks to standard
# output.

moves = int(sys.argv[1])

n = stdio.readInt()
stdio.readInt() # Discard the second int of standard input.

# Read the transition matrix from standard input.
# probs[i][j] is the probability that the surfer moves from
# page i to page j.
probs = stdarray.create2D(n, n, 0.0)
for i in range(n):
    for j in range(n):
        probs[i][j] = stdio.readFloat()

# Use the power method to compute page ranks.
ranks = stdarray.create1D(n, 0.0)
ranks[0] = 1.0
for i in range(moves):
    # Compute effect of next move on page ranks.
    newRanks = stdarray.create1D(n, 0.0)
    for j in range(n):
        # New rank of page j is dot product
        # of old ranks and column j of probs.
        for k in range(n):
            newRanks[j] += ranks[k] * probs[k][j]
    ranks = newRanks

# Write the page ranks.
for rank in ranks:
    stdio.writef("%8.5f", rank)
stdio.writeln()
```

In [4]:
! python transition.py < tiny.txt | python markov.py 20
! python transition.py < tiny.txt | python markov.py 40

 0.27245 0.26515 0.14669 0.24764 0.06806
 0.27303 0.26573 0.14618 0.24723 0.06783
